# Combine nmaps

#### In this notebook, the goal is to import the neuromaps data for glu-related maps and generate some binarized masks.

1. Import packages and maps
2. Combine receptor maps of interest

## Load Packages

In [1]:
import pandas as pd
import nilearn
from nilearn.datasets import fetch_atlas_schaefer_2018
from nilearn.image import binarize_img
from neuromaps.parcellate import Parcellater
import warnings
from neuromaps.datasets import fetch_fslr, fetch_annotation, available_annotations
from neuromaps import datasets, images, nulls, resampling, stats, transforms, parcellate
from neuromaps.parcellate import Parcellater 
from neuromaps.resampling import resample_images
from neuromaps.images import dlabel_to_gifti
import neuromaps
import nibabel as nib
import os
import numpy as np
import mayavi
from PIL import Image
import hcp_utils as hcp
import nilearn.plotting as plotting
import warnings
import nibabel as nib
import abagen ## This is important for certification purposes 

warnings.filterwarnings('ignore')


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


ModuleNotFoundError: No module named 'abagen'

## Import Data and Set Variables

In [3]:
# List annotations of interest. 
ANNOTATIONS = [
    ('dukart2018', 'flumazenil', 'MNI152', '3mm'),   # GABA
    ('rosaneto', 'abp688', 'MNI152', '1mm'),         # mGluR1
    ('dubois2015', 'abp688', 'MNI152', '1mm'),       # mGluR2
    ('smart2019', 'abp688', 'MNI152', '1mm')         # mGluR3
]
# Import MNI brain mask
brainmask = nib.load(mni152['brainmask'])

# Generate an empty list to add transformed maps
nmaps_transformed = []
nmaps_transformed_array = []

# Set paths
outpath = "/Users/pecsok/projects/Neuromaps/pecsok_pfns/neuromaps/data/maps/"
nmda_path = "/Users/pecsok/projects/Neuromaps/pecsok_pfns/neuromaps/data/maps/PET_nifti_images/"

# Notes:
#  1. NMDA is not in available annotations list but was copied from Hansen git repo
#       Already in MNI 152 at 1mm resolution. transfer to outt
#  2. We are excluding GABA map ('norgaard2021', 'flumazenil', 'MNI152', '1mm') 
#      Only has cortical data. Consider re-adding for later analyses or future studies. 


NameError: name 'mni152' is not defined

In [33]:
# Set path for downloading neuromaps data
data_dir = '/Users/pecsok/projects/Neuromaps'


### Load in neuromaps of interest

In [24]:
for (src, desc, space, den) in ANNOTATIONS:
    img = datasets.fetch_annotation(source=src, desc=desc,
                                    space=space, den=den)
    data_transformed = transforms.mni152_to_mni152(img, target='1mm',
                                                   method='linear')
    data_transformed_array = data_transformed.get_fdata()
    # i'm adding a 4th dimension to the dukart map as the transformed data has
    # shape of (193, 229, 193) whereas the norgaard map is (193, 229, 193, 1) **Uncomment lines below if noorgard is added back in.
    # if len(data_transformed.shape) != 4:
    #    data_transformed = data_transformed[..., np.newaxis]
    nmaps_transformed.append(data_transformed)
    nmaps_transformed_array.append(data_transformed_array)

### Combine and save niftis

In [31]:
# Save GABA nifti
GABA_data = nmaps_transformed[0]
GABA_array = nmaps_transformed_array[0]
nib.save(nib.Nifti1Image(GABA_array,GABA_data.affine),outpath +'GABA.nii.gz')

# Sanity check:
print(GABA_array.shape)

(193, 229, 193)


In [37]:
# Combine and save mGluR niftis
mGluR1_data = nmaps_transformed[1]
mGluR1_array = nmaps_transformed_array[1]
mGluR2_data = nmaps_transformed[2]
mGluR2_array = nmaps_transformed_array[2]
mGluR3_data = nmaps_transformed[3]
mGluR3_array = nmaps_transformed_array[3]

avg_img = ((22*mGluR1_array)+(28*mGluR2_array)+(73*mGluR3_array))/(22+28+73)
nib.save(nib.Nifti1Image(avg_img,mGluR2_data.affine),outpath +'mGluR.nii.gz')

# Sanity check:
print(mGluR1_array.shape)
print(mGluR2_array.shape)
print(mGluR3_array.shape)

nilearn.masking.apply_mask(imgs, mask_img, dtype='f', smoothing_fwhm=None, ensure_finite=True)[source]
mGluR1_data = nmaps_transformed[1]


(193, 229, 193)
(193, 229, 193)
(193, 229, 193)


In [36]:
# NMDA nifti is already in MNI 152 at 1mm. Transfer to outpath
nmda = nib.load(nmda_path+'NMDA_ge179_hc29_galovic.nii.gz')
nib.save(nmda,outpath+'NMDA.nii.gz')